In [1]:
import pandas as pd
import random
import folium
from folium import Map
from folium.plugins import HeatMapWithTime
import warnings
warnings.filterwarnings("ignore")

In [2]:
GPS_PATH = "/path/to/gps_data"
METADATA_PATH = "/path/to/metadata"
NETWORK_PATH = "/path/to/network_data"
PROFILE_NAME = " " # choose from config you used for gps generation 

In [3]:
gps_all = pd.read_parquet(GPS_PATH, engine="pyarrow")
metadata_all = pd.read_parquet(METADATA_PATH, engine="pyarrow")
network_all = pd.read_parquet(NETWORK_PATH, engine="pyarrow")

In [4]:
# glimpse of data
display(gps_all.head())
display(metadata_all.head())
display(network_all.head())

,user_id,timestamp,lon,lat,profile_name
0,1,2022-07-22 00:01:00,26.713015,58.388403,tartu_user_employed_walk
1,1,2022-07-22 00:02:00,26.713068,58.388426,tartu_user_employed_walk
2,1,2022-07-22 00:07:00,26.713015,58.388403,tartu_user_employed_walk
3,1,2022-07-22 00:12:00,26.713063,58.388412,tartu_user_employed_walk
4,1,2022-07-22 00:16:00,26.713024,58.388438,tartu_user_employed_walk


,user_id,home_id,work_id,regular_loc_array,network_name,profile_name
0,1,226666275,281487941,"[11224863169, 5875646954, 655255248, 477561132...",tartu_walk,tartu_user_employed_walk
1,2,226914014,30238036,"[10750582260, 1117230039, 683365782, 659347606]",tartu_walk,tartu_user_employed_walk
2,1,30235902,30235860,"[926290307, 3080686, 856490240, 289578348, 448...",tartu_walk,tartu_user_employed_walk_2
3,2,228517750,228682426,"[30222511, 9768818547, 9768818542, 5884520568,...",tartu_walk,tartu_user_employed_walk_2
4,3,1117491464,30231615,"[1173953858, 2167289379, 6535357261, 283250668...",tartu_walk,tartu_user_employed_walk_2


,element_type,osmid,centre_y,name,centre_x,loc_type,network_name
0,node,2345996395,58.373195,TÜ Geograafia osakond,26.715520,hw,tartu_walk
1,way,30190910,58.351225,None,26.683441,hw,tartu_walk
2,way,30221534,58.389387,None,26.735223,hw,tartu_walk
3,way,30221538,58.346422,None,26.717757,hw,tartu_walk
4,way,30221540,58.365181,None,26.699989,hw,tartu_walk


In [5]:
gps_profile = gps_all[gps_all['profile_name']==PROFILE_NAME].reset_index(drop=True)
metadata_profile = metadata_all[metadata_all['profile_name']==PROFILE_NAME].reset_index(drop=True)

In [6]:
random_user_id = random.randint(1, gps_profile['user_id'].nunique())
network_name = metadata_profile['network_name'].unique()[0]
network_profile = network_all[network_all['network_name']==network_name].reset_index(drop=True)
gps_random_user = gps_profile[gps_profile['user_id']==random_user_id].reset_index(drop=True)

In [7]:
gps_random_user['date'] = gps_random_user['timestamp'].dt.date.astype(str) + " (" + gps_random_user['timestamp'].dt.day_name().astype(str) + ")"

In [8]:
grouped_data = gps_random_user.groupby('date')
lat_long_list = [groups[['lat', 'lon']].values.tolist() for key, groups in grouped_data]
date_strings = list(gps_random_user['date'].unique())

In [9]:
# add anchors
metadata_user = metadata_profile[metadata_profile['user_id']==random_user_id]
home_anchor = network_profile[network_profile['osmid']==metadata_user['home_id'].values[0]][['centre_y', 'centre_x']].values.tolist()[0]
work_anchor = network_profile[network_profile['osmid']==metadata_user['work_id'].values[0]][['centre_y', 'centre_x']].values.tolist()[0]
regular_event_anchors = network_profile[network_profile['osmid'].isin(metadata_user['regular_loc_array'].values[0])][['centre_y', 'centre_x']].values.tolist()

In [10]:
event_icons = ['picture-o', 'glass', 'shopping-cart', 'info', 'group', 'coffee']

In [11]:
city_lat = 58.378025
city_lon = 26.728493
m=Map(width=1250, 
      height=500,
      location=[city_lat, city_lon], 
      zoom_start=12, 
      min_zoom=3,
      max_bounds=True)

folium.Marker(
  location=home_anchor,
  popup='home', 
  icon=folium.Icon(color='lightgray', icon='home', prefix='fa')
).add_to(m)

folium.Marker(
  location=work_anchor,
  popup='work', 
  icon=folium.Icon(color='darkblue', icon='briefcase', prefix='fa')
).add_to(m)
for regular_event_anchor in regular_event_anchors:
    folium.Marker(
      location= regular_event_anchor,
      popup='regular event',
      icon=folium.Icon(icon=random.choice(event_icons), prefix='fa')
    ).add_to(m)

In [12]:
title_html = f'''
             <h3 align="center" style="font-size:15px"><b>GPS data visualisation for one user ({PROFILE_NAME})</b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))  
HeatMapWithTime(lat_long_list, radius=10, auto_play=True, index=date_strings, max_opacity=0.5, min_speed=2).add_to(m)
m